In [5]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns    
import pandas as pd
from statistics import mean
from statistics import pstdev

sns.set()
sns.set_style('whitegrid')
sns.set_palette('Set3')

def make_valid_prauc_list(preprocess,lr,df):
    validprauc_list = []
    validprauc_std_list = []
    for epoch in range(100):
        epoch += 1
        Validpraucs = df[(df['preprocess']==preprocess) & (df['lr']==lr) & (df['phase'] == 'valid') & (df['epoch'] == epoch)]['pr-auc'].values
        validprauc_list.append(mean(Validpraucs))
        validprauc_std_list.append(pstdev(Validpraucs))
    return validprauc_list,validprauc_std_list

def write_resultHeader(log_path):
    #CSVファイルのヘッダー記述
    with open(log_path,'w') as f:
        writer = csv.writer(f)
        writer.writerow(['サンプラーの種類','γ','β','学習率','Best Valid Pr-AUC','Epoch数','GPU','実施日時'])
        
def write_results(log_path,result_list):
    with open(log_path,'a') as f:
        writer = csv.writer(f)
        writer.writerow(result_list)

In [8]:
import os

cd = os.getcwd()

result_path = os.path.join(cd,'result.csv')
write_resultHeader(result_path)

result_over_path = os.path.join(cd,'result_over.csv')
write_resultHeader(result_over_path)

result_under_path = os.path.join(cd,'result_under.csv')
write_resultHeader(result_under_path)

In [9]:
import os
from glob import glob

all_directories = glob('*')
all_directories = sorted(all_directories)

start_datetime = '230811-20:22'
start_index = 0

end_datetime = '230816-03:07'
end_index = 0

for i,file in enumerate(all_directories):
    if file == start_datetime:
        start_index = i
    elif file == end_datetime:
        end_index = i
        break
        
print(start_index)
print(end_index)

target_directories = all_directories[start_index:end_index]
print(target_directories)

cnt = 0

for target_directory in target_directories:
    target_path = os.path.join(cd,target_directory,'log.csv')
    df = pd.read_csv(target_path)
    if cnt%3 == 0:
        sampler = 'normal'
    elif cnt%3 == 1:
        sampler = 'over'
    elif cnt%3 == 2:
        sampler = 'under'
    lr = df['lr'][0]
    beta = df['beta'][0]
    gamma = df['gamma'][0]
    model_name = df['model_name'][0]
    preprocess = df['preprocess'][0]
    print(df)
    EPOCH = int(len(df) / 10)
    print(EPOCH)
    valid_prauc_list,validprauc_std_list = make_valid_prauc_list(preprocess,lr,df)
    max_index = np.argmax(valid_prauc_list)

    BestPRAUC = str("{0:.3f}".format(max(valid_prauc_list))) + '±' + str("{0:.3f}".format(validprauc_std_list[max_index]))
    Best_Epoch = max_index
    GPUNo = '2'

    print("{0} {1} {2} {3} {4} {5} {6} {7}".format(sampler,gamma,beta,lr,BestPRAUC,Best_Epoch,GPUNo,target_directory))
    if sampler == 'over':
        write_results(result_over_path,[sampler,gamma,beta,lr,BestPRAUC,Best_Epoch,GPUNo,target_directory])
    elif sampler == 'under':
        write_results(result_under_path,[sampler,gamma,beta,lr,BestPRAUC,Best_Epoch,GPUNo,target_directory])
    elif sampler == 'normal':
        write_results(result_path,[sampler,gamma,beta,lr,BestPRAUC,Best_Epoch,GPUNo,target_directory])
        
    cnt += 1

#後処理
#result_df = pd.read_csv(result_path)
result_over_df = pd.read_csv(result_over_path)

0
179
['230811-20:22', '230811-20:48', '230811-21:14', '230811-21:41', '230811-22:07', '230811-22:35', '230811-23:02', '230811-23:30', '230811-23:57', '230812-00:25', '230812-00:53', '230812-01:21', '230812-01:48', '230812-02:16', '230812-02:44', '230812-03:12', '230812-03:40', '230812-04:08', '230812-04:37', '230812-05:05', '230812-05:32', '230812-06:01', '230812-06:29', '230812-06:58', '230812-07:26', '230812-07:54', '230812-08:22', '230812-08:51', '230812-09:20', '230812-09:49', '230812-10:18', '230812-10:47', '230812-11:16', '230812-11:46', '230812-12:15', '230812-12:44', '230812-13:13', '230812-13:43', '230812-14:12', '230812-14:41', '230812-15:10', '230812-15:40', '230812-16:09', '230812-16:39', '230812-17:08', '230812-17:37', '230812-18:06', '230812-18:36', '230812-19:05', '230812-19:34', '230812-20:03', '230812-20:32', '230812-21:01', '230812-21:31', '230812-22:00', '230812-22:29', '230812-22:59', '230812-23:28', '230812-23:58', '230813-00:27', '230813-00:56', '230813-01:41', '

under 0.5 0.0 0.0001 0.770±0.015 74 2 230811-21:14
    model_name      lr  seed      preprocess sampler  beta  gamma     phase  \
0     ResNet18  0.0005     0  Add_BlackRects   under   0.0    0.5  learning   
1     ResNet18  0.0005     0  Add_BlackRects   under   0.0    0.5     valid   
2     ResNet18  0.0005     0  Add_BlackRects   under   0.0    0.5  learning   
3     ResNet18  0.0005     0  Add_BlackRects   under   0.0    0.5     valid   
4     ResNet18  0.0005     0  Add_BlackRects   under   0.0    0.5  learning   
..         ...     ...   ...             ...     ...   ...    ...       ...   
995   ResNet18  0.0005     0  Add_BlackRects   under   0.0    0.5     valid   
996   ResNet18  0.0005     0  Add_BlackRects   under   0.0    0.5  learning   
997   ResNet18  0.0005     0  Add_BlackRects   under   0.0    0.5     valid   
998   ResNet18  0.0005     0  Add_BlackRects   under   0.0    0.5  learning   
999   ResNet18  0.0005     0  Add_BlackRects   under   0.0    0.5     valid   



under 0.5 0.9 1e-05 0.784±0.022 45 2 230811-23:57
    model_name       lr  seed      preprocess sampler  beta  gamma     phase  \
0     ResNet18  0.00005     0  Add_BlackRects   under   0.9    0.5  learning   
1     ResNet18  0.00005     0  Add_BlackRects   under   0.9    0.5     valid   
2     ResNet18  0.00005     0  Add_BlackRects   under   0.9    0.5  learning   
3     ResNet18  0.00005     0  Add_BlackRects   under   0.9    0.5     valid   
4     ResNet18  0.00005     0  Add_BlackRects   under   0.9    0.5  learning   
..         ...      ...   ...             ...     ...   ...    ...       ...   
995   ResNet18  0.00005     0  Add_BlackRects   under   0.9    0.5     valid   
996   ResNet18  0.00005     0  Add_BlackRects   under   0.9    0.5  learning   
997   ResNet18  0.00005     0  Add_BlackRects   under   0.9    0.5     valid   
998   ResNet18  0.00005     0  Add_BlackRects   under   0.9    0.5  learning   
999   ResNet18  0.00005     0  Add_BlackRects   under   0.9    0.5    

under 0.5 0.99 0.0001 0.789±0.032 90 2 230812-02:44
    model_name      lr  seed      preprocess sampler  beta  gamma     phase  \
0     ResNet18  0.0005     0  Add_BlackRects   under  0.99    0.5  learning   
1     ResNet18  0.0005     0  Add_BlackRects   under  0.99    0.5     valid   
2     ResNet18  0.0005     0  Add_BlackRects   under  0.99    0.5  learning   
3     ResNet18  0.0005     0  Add_BlackRects   under  0.99    0.5     valid   
4     ResNet18  0.0005     0  Add_BlackRects   under  0.99    0.5  learning   
..         ...     ...   ...             ...     ...   ...    ...       ...   
995   ResNet18  0.0005     0  Add_BlackRects   under  0.99    0.5     valid   
996   ResNet18  0.0005     0  Add_BlackRects   under  0.99    0.5  learning   
997   ResNet18  0.0005     0  Add_BlackRects   under  0.99    0.5     valid   
998   ResNet18  0.0005     0  Add_BlackRects   under  0.99    0.5  learning   
999   ResNet18  0.0005     0  Add_BlackRects   under  0.99    0.5     valid   


under 1.0 0.0 1e-05 0.667±0.001 10 2 230812-05:32
    model_name       lr  seed      preprocess sampler  beta  gamma     phase  \
0     ResNet18  0.00005     0  Add_BlackRects   under   0.0    1.0  learning   
1     ResNet18  0.00005     0  Add_BlackRects   under   0.0    1.0     valid   
2     ResNet18  0.00005     0  Add_BlackRects   under   0.0    1.0  learning   
3     ResNet18  0.00005     0  Add_BlackRects   under   0.0    1.0     valid   
4     ResNet18  0.00005     0  Add_BlackRects   under   0.0    1.0  learning   
..         ...      ...   ...             ...     ...   ...    ...       ...   
995   ResNet18  0.00005     0  Add_BlackRects   under   0.0    1.0     valid   
996   ResNet18  0.00005     0  Add_BlackRects   under   0.0    1.0  learning   
997   ResNet18  0.00005     0  Add_BlackRects   under   0.0    1.0     valid   
998   ResNet18  0.00005     0  Add_BlackRects   under   0.0    1.0  learning   
999   ResNet18  0.00005     0  Add_BlackRects   under   0.0    1.0    

...

under 1.0 0.99 1e-05 0.799±0.024 98 2 230815-13:28
    model_name       lr  seed      preprocess sampler  beta  gamma     phase  \
0     ResNet18  0.00005     0  Add_BlackRects    over  0.99    1.0  learning   
1     ResNet18  0.00005     0  Add_BlackRects    over  0.99    1.0     valid   
2     ResNet18  0.00005     0  Add_BlackRects    over  0.99    1.0  learning   
3     ResNet18  0.00005     0  Add_BlackRects    over  0.99    1.0     valid   
4     ResNet18  0.00005     0  Add_BlackRects    over  0.99    1.0  learning   
..         ...      ...   ...             ...     ...   ...    ...       ...   
995   ResNet18  0.00005     0  Add_BlackRects    over  0.99    1.0     valid   
996   ResNet18  0.00005     0  Add_BlackRects    over  0.99    1.0  learning   
997   ResNet18  0.00005     0  Add_BlackRects    over  0.99    1.0     valid   
998   ResNet18  0.00005     0  Add_BlackRects    over  0.99    1.0  learning   
999   ResNet18  0.00005     0  Add_BlackRects    over  0.99    1.0   

under 1.0 0.999 0.0001 0.801±0.017 75 2 230815-16:30
    model_name      lr  seed      preprocess sampler   beta  gamma     phase  \
0     ResNet18  0.0005     0  Add_BlackRects    over  0.999    1.0  learning   
1     ResNet18  0.0005     0  Add_BlackRects    over  0.999    1.0     valid   
2     ResNet18  0.0005     0  Add_BlackRects    over  0.999    1.0  learning   
3     ResNet18  0.0005     0  Add_BlackRects    over  0.999    1.0     valid   
4     ResNet18  0.0005     0  Add_BlackRects    over  0.999    1.0  learning   
..         ...     ...   ...             ...     ...    ...    ...       ...   
995   ResNet18  0.0005     0  Add_BlackRects    over  0.999    1.0     valid   
996   ResNet18  0.0005     0  Add_BlackRects    over  0.999    1.0  learning   
997   ResNet18  0.0005     0  Add_BlackRects    over  0.999    1.0     valid   
998   ResNet18  0.0005     0  Add_BlackRects    over  0.999    1.0  learning   
999   ResNet18  0.0005     0  Add_BlackRects    over  0.999    1.0 

under 2.0 -1.0 1e-05 0.697±0.028 97 2 230815-19:31
    model_name       lr  seed      preprocess sampler  beta  gamma     phase  \
0     ResNet18  0.00005     0  Add_BlackRects    over  -1.0    2.0  learning   
1     ResNet18  0.00005     0  Add_BlackRects    over  -1.0    2.0     valid   
2     ResNet18  0.00005     0  Add_BlackRects    over  -1.0    2.0  learning   
3     ResNet18  0.00005     0  Add_BlackRects    over  -1.0    2.0     valid   
4     ResNet18  0.00005     0  Add_BlackRects    over  -1.0    2.0  learning   
..         ...      ...   ...             ...     ...   ...    ...       ...   
995   ResNet18  0.00005     0  Add_BlackRects    over  -1.0    2.0     valid   
996   ResNet18  0.00005     0  Add_BlackRects    over  -1.0    2.0  learning   
997   ResNet18  0.00005     0  Add_BlackRects    over  -1.0    2.0     valid   
998   ResNet18  0.00005     0  Add_BlackRects    over  -1.0    2.0  learning   
999   ResNet18  0.00005     0  Add_BlackRects    over  -1.0    2.0   

under 2.0 0.9 0.0001 0.802±0.021 35 2 230815-22:33
    model_name      lr  seed      preprocess sampler  beta  gamma     phase  \
0     ResNet18  0.0005     0  Add_BlackRects    over   0.9    2.0  learning   
1     ResNet18  0.0005     0  Add_BlackRects    over   0.9    2.0     valid   
2     ResNet18  0.0005     0  Add_BlackRects    over   0.9    2.0  learning   
3     ResNet18  0.0005     0  Add_BlackRects    over   0.9    2.0     valid   
4     ResNet18  0.0005     0  Add_BlackRects    over   0.9    2.0  learning   
..         ...     ...   ...             ...     ...   ...    ...       ...   
995   ResNet18  0.0005     0  Add_BlackRects    over   0.9    2.0     valid   
996   ResNet18  0.0005     0  Add_BlackRects    over   0.9    2.0  learning   
997   ResNet18  0.0005     0  Add_BlackRects    over   0.9    2.0     valid   
998   ResNet18  0.0005     0  Add_BlackRects    over   0.9    2.0  learning   
999   ResNet18  0.0005     0  Add_BlackRects    over   0.9    2.0     valid   



under 2.0 0.999 1e-05 0.722±0.028 98 2 230816-01:36
    model_name       lr  seed      preprocess sampler   beta  gamma     phase  \
0     ResNet18  0.00005     0  Add_BlackRects    over  0.999    2.0  learning   
1     ResNet18  0.00005     0  Add_BlackRects    over  0.999    2.0     valid   
2     ResNet18  0.00005     0  Add_BlackRects    over  0.999    2.0  learning   
3     ResNet18  0.00005     0  Add_BlackRects    over  0.999    2.0     valid   
4     ResNet18  0.00005     0  Add_BlackRects    over  0.999    2.0  learning   
..         ...      ...   ...             ...     ...    ...    ...       ...   
995   ResNet18  0.00005     0  Add_BlackRects    over  0.999    2.0     valid   
996   ResNet18  0.00005     0  Add_BlackRects    over  0.999    2.0  learning   
997   ResNet18  0.00005     0  Add_BlackRects    over  0.999    2.0     valid   
998   ResNet18  0.00005     0  Add_BlackRects    over  0.999    2.0  learning   
999   ResNet18  0.00005     0  Add_BlackRects    over  0.

In [17]:
result_over_df = pd.read_csv(result_over_path)
result_over_df = result_over_df.sort_values(by='β',ascending=True)
result_over_df.to_csv(result_over_path,index=False,encoding='shift-jis')

In [18]:
result_under_df = pd.read_csv(result_under_path)
result_under_df = result_under_df.sort_values(by='β',ascending=True)
result_under_df.to_csv(result_under_path,index=False,encoding='shift-jis')

In [19]:
result_df = pd.read_csv(result_path)

In [20]:
#この並び順に逆に統一

result_df = result_df.sort_values(by='β',ascending=True)
result_df.to_csv(result_path,index=False,encoding="shift-jis")